In [1]:
import sys
import os
import random
# Counter
from collections import Counter
# Package
import somhos.resources.dataset as rd
import somhos.resources.queries as rq
from somhos.methods.useful import save_pickle, load_pickle
from somhos.config.paths import get_relative_path
from somhos.config.paths import DOCS_SAMPLE_A_SUFFIX, DOCS_SAMPLE_B_SUFFIX
from somhos.config.paths import DOCS_SAMPLES_CONTENT, SAMPLE_PATH, DOCS_SAMPLES_WORD_COUNT
from somhos.config.paths import DOCS_SAMPLES_WORD_DOC_COUNT
from somhos.config.paths import DOCS_SAMPLES_JACCARD_SIM, DOCS_SAMPLES_JACCARD_SIM_UDV
from somhos.config.paths import DOCS_SAMPLES_TF_IDF, KPS_DIRECTORY_INVERSE_SUFFIX

Default path

In [2]:
data_path = "../../src/somhos/resources/aminer/v9gamma"

Load samples

In [3]:
# Load sample A
docs_sample_a_path = get_relative_path(data_path, DOCS_SAMPLE_A_SUFFIX)
docs_sample_a = load_pickle(docs_sample_a_path)
# Load sample B
docs_sample_b_path = get_relative_path(data_path, DOCS_SAMPLE_B_SUFFIX)
docs_sample_b = load_pickle(docs_sample_b_path)

print("Samples size: (%d, %d)" % (len(docs_sample_a), len(docs_sample_b)))

Samples size: (5000, 5000)


Load samples content

In [4]:
docs_samples_content_path = get_relative_path(data_path, DOCS_SAMPLES_CONTENT)
samples_content = load_pickle(docs_samples_content_path)
print("Docs: %d" % len(samples_content))

Docs: 10000


In [5]:
from nltk.metrics.distance import jaccard_distance
from itertools import product
import numpy as np

In [6]:
sample_a_indices = range(0, len(docs_sample_a))
sample_b_indices = range(0, len(docs_sample_b))

def get_bow(index, samples_content):
    return samples_content[index]['bag-of-words']

m_jaccard_sim = np.zeros([len(docs_sample_a), len(docs_sample_b)]) 
for i, j in product(sample_a_indices, sample_a_indices):
    set_a = get_bow(docs_sample_a[i], samples_content)
    set_b = get_bow(docs_sample_b[j], samples_content)
    jaccard_sim = 1 - jaccard_distance(set_a, set_b)
    m_jaccard_sim[i, j] = jaccard_sim

print(m_jaccard_sim)

[[0.03472222 0.0174216  0.02105263 ... 0.03184713 0.01798561 0.04268293]
 [0.04216867 0.0308642  0.02469136 ... 0.04736842 0.02597403 0.05853659]
 [0.02857143 0.         0.03007519 ... 0.00591716 0.0078125  0.03867403]
 ...
 [0.02739726 0.03597122 0.0141844  ... 0.03529412 0.02272727 0.02645503]
 [0.0308642  0.01257862 0.01910828 ... 0.02659574 0.02013423 0.02427184]
 [0.03496503 0.0070922  0.01438849 ... 0.06097561 0.03100775 0.0212766 ]]


In [7]:
docs_samples_jaccard_sim_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM)
save_pickle(m_jaccard_sim, docs_samples_jaccard_sim_path)

In [8]:
n_singularvalues = 1
U, s, V = np.linalg.svd(m_jaccard_sim, full_matrices=False)
D = np.diag(s[:n_singularvalues])
m_udv_jaccard_sim = np.dot(U[:,:n_singularvalues], np.dot(D, V[:n_singularvalues,:]))
print(m_udv_jaccard_sim)

[[0.03389149 0.01443577 0.0261616  ... 0.03203783 0.02327397 0.03159588]
 [0.03742246 0.01593975 0.02888723 ... 0.03537568 0.02569875 0.03488768]
 [0.0201368  0.00857708 0.01554404 ... 0.01903544 0.01382834 0.01877285]
 ...
 [0.04150054 0.01767677 0.03203519 ... 0.03923071 0.02849925 0.03868954]
 [0.03186418 0.01357225 0.02459667 ... 0.0301214  0.02188178 0.02970589]
 [0.04029545 0.01716348 0.03110496 ... 0.03809153 0.0276717  0.03756607]]


In [9]:
docs_samples_jaccard_sim_udv_path = get_relative_path(data_path, DOCS_SAMPLES_JACCARD_SIM_UDV)
save_pickle(m_udv_jaccard_sim, docs_samples_jaccard_sim_udv_path)